In [1]:
import sys
sys.path.insert(1, 'Classes/')
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from multiprocessing import current_process

from ScoreGetter import ScoreGetter
from dataset_utils import checkIfEarlyMidEnd
from dataset_utils import encodeBoard
from dataset_utils import getColumns

In [2]:
df = pd.read_csv('Datasets/raw_dataset_13M.csv')
boards = df['board'].values

#### We load an engine to get a score from the positions.

In [3]:
score_getter = ScoreGetter('/usr/local/bin/stockfish', 'eval', 'go depth 1')

#### We create our dataset by getting an equal number of start, middle and end game positions and encoding them.

In [ ]:
total_size = 500000
batch_size = total_size / 3.0

earlies = []
mids = []
ends = []

current = current_process()
pos = current._identity[0]-1 if len(current._identity) > 0 else 0
pbar = tqdm(total=total_size, desc='Splitting and encoding', position=pos)

for board in boards:
    part = checkIfEarlyMidEnd(board)
    
    if len(earlies) < batch_size and part == "early_game":
        earlies.append(np.append(encodeBoard(board), score_getter.getScore(board)))
        pbar.update(1)
    
    elif len(mids) < batch_size and part == "mid_game":
        mids.append(np.append(encodeBoard(board), score_getter.getScore(board)))
        pbar.update(1)
        
    elif len(ends) < batch_size and part == "end_game":
        ends.append(np.append(encodeBoard(board), score_getter.getScore(board)))
        pbar.update(1)
    
    if len(earlies) >= batch_size and len(mids) >= batch_size and len(ends) >= batch_size:
        break
    
pbar.close()

Splitting and encoding:  48%|████▊     | 242128/500000 [05:42<06:01, 712.47it/s]

In [ ]:
data = earlies + mids + ends
random.shuffle(data)
len(data)

In [ ]:
df = pd.DataFrame(data, columns = np.append(getColumns(), 'cp (Stockfish 13)'))
df.to_csv('Datasets/dataset.csv', index=False)
df.head()

#### We create a test dataset.

In [ ]:
'''
random.shuffle(boards)

test_size = 200000
tests = []

current = current_process()
pos = current._identity[0]-1 if len(current._identity) > 0 else 0
pbar = tqdm(total=test_size, desc='Encoding', position=pos)


for i in range(test_size):
    tests.append(np.append(encodeBoard(boards[i]), score_getter.getScore(boards[i])))
    pbar.update(1)
pbar.close()

len(tests)
'''

In [ ]:
'''
df = pd.DataFrame(tests, columns = np.append(getColumns(), 'cp (Stockfish 13)'))
df.to_csv('Datasets/test_dataset.csv', index=False)
df.head()
'''